In [103]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, RobustScaler

In [ ]:
train = pd.read_csv(r"\playground-series-s6e2\train.csv")
train = train.set_index('id')
le = LabelEncoder()
train['Heart Disease'] = le.fit_transform(train['Heart Disease'])
y = train['Heart Disease']
train = train.drop(columns='Heart Disease')

test = pd.read_csv(r"\playground-series-s6e2\test.csv")
test = test.set_index('id')

# Basic check

In [105]:
y.value_counts(normalize=True)

Heart Disease
0    0.55166
1    0.44834
Name: proportion, dtype: float64

In [106]:
import EDA
eda = EDA.BasicEDA(train)
print(eda.check_missing_values())
print(eda.check_duplicate())
print(eda.check_unique_count())

No Missing value
No duplicate
                    Feature  Unique  Sample_size  Cardinality_Ratio
4               Cholesterol     150       630000            0.02381
7                    Max HR      93       630000            0.01476
3                        BP      66       630000            0.01048
9             ST depression      66       630000            0.01048
0                       Age      42       630000            0.00667
11  Number of vessels fluro       4       630000            0.00063
2           Chest pain type       4       630000            0.00063
10              Slope of ST       3       630000            0.00048
6               EKG results       3       630000            0.00048
12                 Thallium       3       630000            0.00048
1                       Sex       2       630000            0.00032
8           Exercise angina       2       630000            0.00032
5              FBS over 120       2       630000            0.00032


In [107]:
num_fea = ['Age','BP','Cholesterol','Max HR','ST depression']
cate_fea = list(set(train.columns).difference(num_fea))
eda.skewness_kurtosis(num_fea)

,feature,count,mean,std,min,25%,50%,75%,max,range,iqr,skewness,kurtosis
3,Max HR,630000.0,152.816763,19.112927,71.0,142.0,157.0,166.0,202.0,131.0,24.0,-0.754556,0.203852
0,Age,630000.0,54.136706,8.256301,29.0,48.0,54.0,60.0,77.0,48.0,12.0,-0.164011,-0.570157
2,Cholesterol,630000.0,245.011814,33.681581,126.0,223.0,243.0,269.0,564.0,438.0,46.0,0.273150,0.068237
1,BP,630000.0,130.497433,14.975802,94.0,120.0,130.0,140.0,200.0,106.0,20.0,0.629283,0.933183
4,ST depression,630000.0,0.716028,0.948472,0.0,0.0,0.1,1.4,6.2,6.2,1.4,1.328429,1.374855


In [108]:
train['ST_dpe_is_zero'] = (train['ST depression'] == 0).astype(int)
train['ST depression'] = np.log(train['ST depression']+1)
train['cholesterol_high'] = (train['Cholesterol'] > train['Cholesterol'].quantile(.9)).astype(int)

In [109]:
cate_fea

['Number of vessels fluro',
 'Slope of ST',
 'EKG results',
 'Thallium',
 'Exercise angina',
 'Sex',
 'FBS over 120',
 'Chest pain type']

In [110]:
eda.Categorical_Dis(cate_fea)

,feature,gini,entropy
6,FBS over 120,0.147179,0.402134
4,Exercise angina,0.397600,0.846760
5,Sex,0.407778,0.862526
2,EKG results,0.501924,1.019351
3,Thallium,0.497098,1.079860
1,Slope of ST,0.510557,1.122396
0,Number of vessels fluro,0.461556,1.265748
7,Chest pain type,0.612719,1.581770


In [111]:
train['hard1'] = train['Age'] * train['FBS over 120']
train['hard2'] = train['Cholesterol'] * train['FBS over 120']

# Train

In [112]:
import lightgbm as lgb
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
oof = np.zeros(len(train))

def objective(trial):
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'verbosity': -1,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 16, 256),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0)
    }
    
    fold_aucs = []
    for train_idx, valid_idx in kf.split(train, y):
        X_train, X_valid = train.iloc[train_idx], train.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        
        train_data = lgb.Dataset(X_train, label=y_train)
        valid_data = lgb.Dataset(X_valid, label=y_valid,reference=train_data)
        
        model = lgb.train(
            params,
            train_data,
            valid_sets=[valid_data],
            num_boost_round=1000,    
            callbacks=[
                lgb.early_stopping(stopping_rounds=50),  # 改用 callback 形式
                lgb.log_evaluation(period=0)  # period=0 相当于 verbose_eval=False
            ]
        )
        preds = model.predict(X_valid)
        fold_aucs.append(roc_auc_score(y_valid, preds))
    
    return np.mean(fold_aucs)

study = optuna.create_study(direction='maximize', study_name='lgb_baseline')
study.optimize(objective, n_trials=20)

[I 2026-02-01 22:53:03,343] A new study created in memory with name: lgb_baseline


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[191]	valid_0's auc: 0.955587
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[171]	valid_0's auc: 0.954571
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[178]	valid_0's auc: 0.955371
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[189]	valid_0's auc: 0.955024
Training until validation scores don't improve for 50 rounds


[I 2026-02-01 22:53:17,230] Trial 0 finished with value: 0.9552739040374327 and parameters: {'learning_rate': 0.11230404499759662, 'num_leaves': 93, 'max_depth': 9, 'min_child_samples': 62, 'subsample': 0.8631393922830213, 'colsample_bytree': 0.5317337830989044, 'reg_alpha': 7.055293715366454, 'reg_lambda': 4.495551116317049}. Best is trial 0 with value: 0.9552739040374327.


Early stopping, best iteration is:
[182]	valid_0's auc: 0.955817
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[747]	valid_0's auc: 0.955741
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[550]	valid_0's auc: 0.954729
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[598]	valid_0's auc: 0.955525
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[653]	valid_0's auc: 0.955117
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[706]	valid_0's auc: 0.955955


[I 2026-02-01 22:54:02,136] Trial 1 finished with value: 0.9554133483075018 and parameters: {'learning_rate': 0.04626986965181246, 'num_leaves': 222, 'max_depth': 6, 'min_child_samples': 78, 'subsample': 0.5157531931463704, 'colsample_bytree': 0.5549777666485125, 'reg_alpha': 6.721730229564979, 'reg_lambda': 1.1998701191116412}. Best is trial 1 with value: 0.9554133483075018.


...

[I 2026-02-01 23:03:19,470] Trial 17 finished with value: 0.9548697125272746 and parameters: {'learning_rate': 0.030761392710064752, 'num_leaves': 256, 'max_depth': 12, 'min_child_samples': 71, 'subsample': 0.7370192751682683, 'colsample_bytree': 0.8016407193087124, 'reg_alpha': 4.913408578475066, 'reg_lambda': 2.177267607652761}. Best is trial 13 with value: 0.9554825560138003.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[388]	valid_0's auc: 0.95566
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[320]	valid_0's auc: 0.954632
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[337]	valid_0's auc: 0.955408
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[385]	valid_0's auc: 0.955082
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[393]	valid_0's auc: 0.955876


[I 2026-02-01 23:03:37,011] Trial 18 finished with value: 0.9553316882663893 and parameters: {'learning_rate': 0.1505190886607639, 'num_leaves': 119, 'max_depth': 4, 'min_child_samples': 87, 'subsample': 0.6929969004130206, 'colsample_bytree': 0.9172314030325278, 'reg_alpha': 2.7724723316581326, 'reg_lambda': 5.4831424673760605}. Best is trial 13 with value: 0.9554825560138003.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[606]	valid_0's auc: 0.95565
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[494]	valid_0's auc: 0.954595
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[607]	valid_0's auc: 0.955443
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[603]	valid_0's auc: 0.955007
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[760]	valid_0's auc: 0.955831


[I 2026-02-01 23:04:07,379] Trial 19 finished with value: 0.9553051934755136 and parameters: {'learning_rate': 0.06020512210485548, 'num_leaves': 199, 'max_depth': 5, 'min_child_samples': 52, 'subsample': 0.5765625096703267, 'colsample_bytree': 0.8561126512163446, 'reg_alpha': 3.3410985903729458, 'reg_lambda': 3.020046076312271}. Best is trial 13 with value: 0.9554825560138003.


In [114]:
train_data = lgb.Dataset(train, label=y)
model = lgb.train(
            study.best_params,
            train_data,
            num_boost_round=1000,    
            callbacks=[
                lgb.log_evaluation(period=0)  
            ]
        )

# Check distribution

In [6]:
import CheckData
check = CheckData.ShiftData(train, test)

In [62]:
check.check_num_shift(num_fea)

,feature,wasserstein_dist,ks_pvalue
4,ST depression,0.003037,0.164060
1,BP,0.067123,0.256862
0,Age,0.029071,0.339312
3,Max HR,0.059884,0.618227
2,Cholesterol,0.069201,0.854634


In [8]:
check.check_cate_shift(cate_fea)

,feature,chi2_pvalue,kl_divergence,js_divergence,n_unique_train,n_unique_test
6,FBS over 120,0.920982,2.829684e-08,7.075078e-09,2,2
3,Thallium,0.904966,5.292115e-07,1.322044e-07,3,3
4,Exercise angina,0.365955,2.172984e-06,5.434371e-07,2,2
2,EKG results,0.313603,6.123446e-06,1.532237e-06,3,3
5,Sex,0.128680,6.134793e-06,1.532842e-06,2,2
7,Chest pain type,0.289258,9.938104e-06,2.483804e-06,4,4
0,Number of vessels fluro,0.199467,1.227232e-05,3.070785e-06,4,4
1,Slope of ST,0.020893,2.044430e-05,5.113684e-06,3,3


# Fit

In [115]:
test['ST_dpe_is_zero'] = (test['ST depression'] == 0).astype(int)
test['ST depression'] = np.log(test['ST depression']+1)
test['cholesterol_high'] = (test['Cholesterol'] > test['Cholesterol'].quantile(.9)).astype(int)
test['hard1'] = test['Age'] * test['FBS over 120']
test['hard2'] = test['Cholesterol'] * test['FBS over 120']

In [116]:
preds = model.predict(test)

In [117]:
test['Heart Disease'] = (preds >0.5).astype(int)

In [118]:
test = test.reset_index()

In [ ]:
test[['id','Heart Disease']].set_index('id').to_csv(r'\playground-series-s6e2\submission.csv')